In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, BatchNormalization

In [39]:
# 먼저 영어-프랑스 데이터 를 받아온다.
with open('/content/drive/MyDrive/Colab Notebooks/fra-eng/fra.txt', 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')

input_texts = []
target_texts = []
target_text = ""
for line in lines[:3000]:
    input_text, target_text, des = line.split('\t')
    input_texts.append(input_text)
    # Each '\t' and '\n' is used as 'start' and 'end' sequence character
    target_text = '\t' + target_text + '\n'   # target 텍스트는 정답의 시작/끝을 알리는 문자로 '탭' 사용
    target_texts.append(target_text)

In [40]:
# 각 데이터의 형태는 (텍스트 개수, 최대 문장 길이, 사용되는 토큰 수)
latent_dim = 256
input_characters = set()
target_characters = set()
# 각 데이터를 구성하는 문자를 ch 에 저장 -> characters 집합에 없으면 저장하는 방식
for input_text, target_text in zip(input_texts, target_texts):
    for ch in input_text:
        if ch not in input_characters:
            input_characters.add(ch)
    for ch in target_text:
        if ch not in target_characters:
            target_characters.add(ch)

input_characters = sorted(list(input_characters))   # input을 구성하는 문자를 정렬된 list로
target_characters = sorted(list(target_characters))  # target을 구성하는 문자를 정렬된 list 로
num_encoder_tokens = len(input_characters)    # input 을 이루는 문자의 개수 (영어 -> 알파벳 개수)
num_decoder_tokens = len(target_characters)   # target 을 이루는 문자의 개수 
max_encoder_seq_length = max([len(txt) for txt in input_texts])   # input 에서 가장 긴 단어의 길이
max_decoder_seq_length = max([len(txt) for txt in target_texts])  # target 에서 가장 긴 단어의 길이

# {id:문자} 를 딕셔너리 형태로 저장
input_token_index = { char: id for id, char in enumerate(input_characters)}   
target_token_index = { char: id for id, char in enumerate(target_characters)}

# 인코더/디코더에 사용할 데이터 형태 지정 & 초기화
encoder_input_data = np.zeros(shape=(len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros(shape=(len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros(shape=(len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

# 각 단어 one-hot 벡터화 -> 각 단어가 벡터 형태로 encoder 에 들어가게 된다.
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for j, ch in enumerate(input_text):
        encoder_input_data[i, j, input_token_index[ch]] = 1.
    for j, ch in enumerate(target_text):
        decoder_input_data[i, j, target_token_index[ch]] = 1.
        if j > 0:
            decoder_target_data[i, j-1, target_token_index[ch]] = 1.

In [41]:
# a part of encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens), name='encoder_input')
encoder = LSTM(latent_dim, return_sequences=True, return_state=True, name='encoder')
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# a part of decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_input')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
batchNorm = BatchNormalization()
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(batchNorm(decoder_outputs))

# a model to train
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [42]:
# 학습
batch_size=64
epochs=100
optimizer='rmsprop'
loss='categorical_crossentropy'
# load_model_path=None, 
save_model_path='/content/drive/MyDrive/Colab Notebooks'
 
# if not load_model_path == None:
#     load_model(load_model_path)
model.compile(optimizer, loss)
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
          batch_size = batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save(save_model_path)


Epoch 1/100
37/38 [============================>.] - ETA: 0s - loss: 1.2229

KeyboardInterrupt: ignored

In [43]:
# 추론 모델 생성
# encoder model to decode
encoder_model = Model(encoder_inputs, encoder_states)

# decoder model to decode
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [49]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [50]:
def decode_sequence(input_seq):
    # Encode the input as state vectors. [state_h, state_c]
    # 입력 문장을 인코딩
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    # 디코더의 입력으로 쓸 단일 문자
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    # 첫 입력은 시작 문자 '\t'
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 점점 디코드된 문자열을 추가해나감.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # 이전의 출력/상태를 디코더에 넣어서 새로운 출력/상태를 얻는다.
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        # 원 핫 인코딩 출력을 실제 문자로 반환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        # 종료 문자가 나왔거나, 문장 길이가 한계를 넘으면 종료한다.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
               stop_condition = True

        # Update the target sequence (of length 1).
        # 디코더의 다음 입력으로 쓸 데이터 갱신
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
         
    return decoded_sentence

In [ ]:
# def translate(self, input_seq):
#     reverse_target_char_index = dict((i, char) for char, i in self.target_token_index.items())
#     # 입력을 모델에 주입할 수 있도록 벡터화한다.
#     input_vec = np.zeros(shape=(len(input_seq), self.max_encoder_seq_length,                                        num_encoder_tokens), dtype='float32')
#     for i, txt in enumerate(input_seq):
#         for j, ch in enumerate(txt):
#             input_vec[i, j, self.input_token_index[ch]] = 1.

#     decoded_sequences = []
#     for seq_idx in range(input_vec.shape[0]):
#         decoded_sequences.append(decode_sequence(input_vec[seq_idx: seq_idx+1]))

#     return decoded_sequences

In [51]:
for seq_index in range(30):
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('"{}" -> "{}"'.format(input_texts[seq_index], decoded_sentence.strip()))

"Go." -> "'aisis e ai."
"Go." -> "'aisis e ai."
"Go." -> "'aisis e ai."
"Hi." -> "'aissste en."
"Hi." -> "'aissste en."
"Run!" -> "'aissste en."
"Run!" -> "'aissste en."
"Run!" -> "'aissste en."
"Run!" -> "'aissste en."
"Run!" -> "'aissste en."
"Run!" -> "'aissste en."
"Run!" -> "'aissste en."
"Run!" -> "'aissste en."
"Run." -> "'aissste en."
"Run." -> "'aissste en."
"Run." -> "'aissste en."
"Run." -> "'aissste en."
"Run." -> "'aissste en."
"Run." -> "'aissste en."
"Run." -> "'aissste en."
"Run." -> "'aissste en."
"Who?" -> "'aissste en."
"Wow!" -> "'assssteis e."
"Fire!" -> "'aissste en."
"Help!" -> "'asssteste ai."
"Jump!" -> "'asssteste ai."
"Jump." -> "'assssteis e."
"Stop!" -> "'aissste en."
"Stop!" -> "'aissste en."
"Stop!" -> "'aissste en."
